In [3]:
import pandas as pd
from taxipred.utils.constants import DATA_PATH

df = pd.read_csv(DATA_PATH / "taxi_cleaned.csv")

df.sample(10)

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
152,17.56,Evening,Weekday,1.0,Low,Clear,3.22,1.22,0.11,103.66,36.0458
193,47.07,Morning,Weekday,4.0,Medium,Rain,2.06,1.86,0.15,71.04,100.2662
133,48.22,Night,Weekday,3.0,Medium,Clear,4.96,1.99,0.22,19.21,105.1440
317,25.11,Afternoon,Weekday,4.0,Medium,Clear,3.66,1.34,0.25,88.10,59.3324
428,8.43,Afternoon,Weekday,4.0,Low,Clear,2.36,0.88,0.13,57.18,NaN
159,18.65,Afternoon,Weekday,1.0,Medium,Clear,4.75,0.98,0.36,87.41,54.4946
521,24.88,Morning,Weekend,2.0,High,Clear,3.40,1.54,0.20,68.14,55.3432
4,41.79,Night,Weekend,3.0,High,Clear,4.60,1.77,0.11,86.95,88.1328
155,23.81,Morning,Weekend,1.0,Medium,Rain,2.81,1.91,0.24,62.02,63.1719
452,46.78,Afternoon,Weekend,3.0,Medium,Clear,3.13,1.75,0.17,34.03,90.7801
